`04-db-rag.ipynb`

In [ ]:
%pip install -q langchain langchain-community sqlalchemy langchain_openai

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.utilities import SQLDatabase
# 현재 사용중인 DB는 sqlite
db = SQLDatabase.from_uri('sqlite:///lottery_results.db')
print(db.get_usable_table_names())
db.run('SELECT * FROM lottery_results LIMIT 5')

['lottery_results']


"[(1, '2002-12-07', 3681782000, 0, 863604600, 0, 10, 23, 29, 33, 37, 40, 16), (2, '2002-12-14', 4904274000, 2002006800, 0, 1, 9, 13, 21, 25, 32, 42, 2), (3, '2002-12-21', 4729342000, 2000000000, 0, 1, 11, 16, 19, 21, 27, 31, 30), (4, '2002-12-28', 5271464000, 0, 1267147200, 0, 14, 27, 30, 31, 40, 42, 2), (5, '2003-01-04', 6277102000, 0, 3041094900, 0, 16, 24, 29, 40, 41, 42, 3)]"

In [32]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain

llm = ChatOpenAI(model='gpt-3.5-turbo')

In [6]:
# 자연어를 기반으로 SQL을 만들어주는 체인
chain = create_sql_query_chain(llm, db)

In [7]:
res = chain.invoke({'question': 'lotto_results 테이블에 데이터가 몇개 있어?'})

In [8]:
print(res)
db.run(res)

SELECT COUNT("drwNo") AS total_rows FROM lottery_results;


'[(1128,)]'

In [9]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [26]:
llm = ChatOpenAI(model='gpt-3.5-turbo')

from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

# 자연어 => SQL 생성
make_query = create_sql_query_chain(llm, db)
# 생성된 SQL을 실행
execute_query = QuerySQLDataBaseTool(db=db)

chain = make_query | execute_query

chain.invoke({'question': 'lotto_results 테이블에 데이터가 몇개 있어?'})

'[(1128,)]'

In [34]:
from operator import itemgetter
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


answer_prompt = PromptTemplate.from_template(
        """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables. 

There is only one table named lottery_results in DB.
These are column explanation.

drwNo: Lottery Draw Round Number
drwNoDate: Date of Lottery Draw Round
totSellamnt: Total Sales Amount
firstAccumamnt: Sum of 1st Winning Amount
firstPrzwnerCo: Sum of 1st Winners.
firstWinamnt: Winning amount for each 1st place winner
drwtNo1: Drawn Number
drwtNo2: Drawn Number
drwtNo3: Drawn Number
drwtNo4: Drawn Number
drwtNo5: Drawn Number
drwtNo6: Drawn Number
bnusNo: Bonus Number

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

# 자연어질문 => SQL => 실행한 결과를 => LLM이 문장으로 조합해서 응답
chain = (
    # 자연어 => SQL => 실행결과
    RunnablePassthrough.assign(query=make_query).assign(
        result=itemgetter('query') | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

In [35]:
res = chain.invoke({'question': '1등한 사람이 가장 많은 돈을 받아간 회차와 액수를 말해줘'})

In [36]:
res

'1등한 사람이 가장 많은 돈을 받아간 회차는 19회차이며 액수는 40,722,959,400원 입니다.'